<a href="https://colab.research.google.com/github/namoshi/mytest/blob/master/tf_iris_logreg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#
#   iris_lr_softmax.py
#       date. 5/6/2016
#       IRIS data set classification
#

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn import datasets


def prep_data(train_siz=120, test_siz=30):
    '''
      class: 
        1. Iris Setosa, 2. Iris Versicolor, 3. Iris Virginica
    '''
#    cols = ['sepal_len', 'sepal_wid', 'petal_len', 'petal_wid', 'class']  
#    iris_df = pd.read_csv('./iris.csv', header=True, names=cols)
    
    # Encode class 
    class_name = ['setosa', 'versicolor', 'virginica']
#    iris_df['iclass'] = [class_name.index(class_str) 
#                            for class_str in iris_df['class'].values]
  
    iris = datasets.load_iris()
    
    iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
    iris_df['target'] = iris.target_names[iris.target]
    iris_df.head()
    
    # Random Shuffle before split to train/test
    data_len = len(iris_df)
    orig = np.arange(data_len)
    perm = np.copy(orig)
    np.random.shuffle(perm)
    iris=iris_df[['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)', 'target']].values
    iris[orig, :] = iris[perm, :]
    
    # generate onehot label data
    label = np.zeros((data_len, 3), dtype=np.float32)
    for i in range(data_len):
      for iclass in range(3):
#        print('i=', i, 'iclass=', iclass, 'iris[i, -1]=',iris[i, -1])
        if (iris[i, -1] == class_name[iclass]):
          label[i, iclass] = 1.0
#      print(label[i])
#    print(label)
    
    # Split dataset
    trX = iris[:train_siz, :-1]
    teX = iris[train_siz:, :-1]
    trY = label[:train_siz, :]
    teY = label[train_siz:, :]
    
    return trX, trY, teX, teY
    
def linear_model(X, w, b):
    output = tf.matmul(X, w) + b

    return output

if __name__ == '__main__':
    tr_x, tr_y, te_x, te_y = prep_data()

    # Variables
    x = tf.compat.v1.placeholder(tf.float32, [None, 4])
    y_ = tf.compat.v1.placeholder(tf.float32, [None, 3])
    
    w = tf.Variable(tf.random_normal([4, 3], mean=0.0, stddev=0.005))
    b = tf.Variable(tf.zeros([3]))

    y_pred = linear_model(x, w, b)
    y_pred_softmax = tf.nn.softmax(y_pred)   # for prediction

    loss = -tf.reduce_sum(y_*tf.log(y_pred_softmax))
    train_step = tf.train.GradientDescentOptimizer(0.001).minimize(loss)

    correct_prediction = tf.equal(tf.argmax(y_pred_softmax, 1), 
        tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    # Train
    init = tf.global_variables_initializer()

    with tf.Session() as sess:
        sess.run(init)

        print('Training...')
        for i in range(10001):
            batch_xs, batch_ys = tr_x, tr_y
            train_step.run({x: batch_xs, y_: batch_ys})                 
        
            if i % 1000 == 0:
                fd_train = {x: batch_xs, y_: batch_ys}
                loss_step = loss.eval(fd_train)
                train_accuracy = accuracy.eval(fd_train)
                print('  step, loss, accurary = %6d: %8.3f,%8.3f' % (i, 
                                                loss_step, train_accuracy))
        
        # Test trained model
        fd_test = {x: te_x, y_: te_y}
        print('accuracy = %10.4f' % accuracy.eval(fd_test))

RuntimeError: ignored